In [1]:

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
import torch

# 确保PyTorch能够检测到GPU

In [2]:

import json
import time
import re
import itertools
import numpy as np
import pandas as pd
import datetime as DT
from tqdm import tqdm
from datetime import datetime
from scipy.optimize import curve_fit
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

In [3]:

# Load pretrained LLM
QWEN2d5_7B_INST = "/home/peizhengqi/Qwen/Qwen2.5-7B-Instruct-AWQ"
QWEN2d5_32B_INST_AWQ = "/home/peizhengqi/Qwen/Qwen2.5-32B-Instruct-AWQ"
QWEN2d5_72B_INST_AWQ = "/home/peizhengqi/Qwen/Qwen2.5-72B-Instruct-AWQ"

#model_7Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
#tokenizer_7Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

model_32Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_32B_INST_AWQ, device_map="auto")
tokenizer_32Bn = AutoTokenizer.from_pretrained(QWEN2d5_32B_INST_AWQ, device_map="auto")

#model_72Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")
#tokenizer_72Bn = AutoTokenizer.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")

OSError: Incorrect path_or_model_id: '/home/peizhengqi/Qwen/Qwen2.5-32B-Instruct-AWQ'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:

from transformers import TextStreamer

streamer = TextStreamer(tokenizer_32Bn, skip_prompt=True, skip_special_tokens=True)

In [ ]:

sys_prompt1 = """
    You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible, while being safe. 
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    Please ensure that your responses are socially unbiased and positive in nature.
    If you don't know the answer to a question, please don't share false information.
"""


def message_to_response_via_LLM(model, tokenizer, messages, max_new_tokens=256):

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.01, streamer=streamer)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    

    return response

In [ ]:

query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "TAATAAGTTCGATAAGAGATTGAACACCAATGATGATAGGATGATGGAGATGACCCTTCAATAGATATCATCTATATTAAGATATAAAGAAATTAATAAAATATAAATAATAATATAATTTTATGTATCACATTGCACCTAATACTTGTATGAGTTGTATCCAACGCCATGAAACAAAATGTGTCAGATATCAGTTAGATAAGATTCAGTTCAGCAGGCAGGTTGGATAAGAGTATTGCTTATGTATATCTTAAAAGAAAGTTTCCTATTTGACAGAATGTGGGTGTTTTTGTCCACCTAGGAGCTTTTTACCATGTAATCTGACATAGGCTTTATGATCTGCACTAATCTTTTGAAACTTCATTTTGTCACATGTTAATAAGTTGAAAAAGAAAATATTGTTTTAGGAGCTAAAGATGAATAAGCAGCTAAAGGAAAGTTATATAGAATTAGTAGTAAAAGAGAAGAGTGTAAACCTATTTGCTATTTGCTTCAGTACTCAAGTGCTGGTTCCGTTACCAAATTTCAGTGGTTTTGGTTGAATATGTAAGTCAGTGGAAGTGGTGCTGAATAGTGGAGGGAATGTTTGGAGTACATGATTTGAAAAGGGGAGAATGATTAAAAGAAATAGAAAATTGACTGACCAATATTGAAGGTTTCATAGGAATGTTGAATGTTAGTAAATTGGAAAGATGTTACATACCATTCATCTTATTAGAAGCATAACTGTTCCTTTTTGTATATTTTCTGTTACCTTCCTTTCCTTCAAAATATGGCCTTTGAATTTTGAACCATTAACTGAGTTGGTATAGTGGTAAAGTCATGTAGATTTTTGTTTTCGTTTTGTGGGGATATGAATGAGAAAAATCCTTACAACCAGACAAAGCTGTGGCTCAGAAGAAAGAGAAACATGCCTCACTTCTAGAATCACTGGAACTAACATCTTGTAAACAAACTGTTATACCAAACTTTCCTAAGGTGCAATTCTCAACCATTAACCAATTTCATCTCAAACAACACTATCAGTTCTTCAGTCATGAACTTCCCATACTGAAAATTTGTCACCTATTTCTTTTATAGGCAGTGGCTTCTTATGAGAGTCACACTCTCTGAACTTCTAATGTAAAACTTGACTTTTCATTTTGTGCTTTATAATTGTTATCTCATGCATTGCCCTTTTAAGATAAGGTTGTTGTGCATTGTTTGGTTGAGTACAACGATTTACTGATGACCATAACTATTTGTGGATCTTATATTTTCAAAAGTGTATGCTTGGTCATTAATCAACCAAGCTTGATAAGCTCCATCACCTACCATCCCCTTCTATGCCATATACACATTTTGCTTATCTACATTCCTACTGTTACTCTCAAACAAGTCTAACAAATAATAAGTGAAGTTAATGAACTTTTTTAGTAATATTATTATAAGGATTGGTCAATGTAATGGTGAACAGAGAAGCATGCTTTCAGCTACCCAACAAAATGGATAAGAGTGTCAGTTGATATGGTTCTCTTGTTTTTTTTTTTTCAATGCAAAATCACCTTTAAAATATAAAAGATCACTGTAACAAAGGGACAAAGGGTTTTCACCTAACCCATGCATGAGATATGATGGTCACAAAATATGTTAGGTTTAATATGGAATGAGGGCACAGTACAAACCACTCACAAATAATATCAAACTCCACCACCATCACACATTTTACGTTCTTCCAAGGAAGAGATAAGATAATGAAGCCTCCTCCACGTGTCACTTCCACATGGTACCTAACGATAAGGCTACCATTCAAAATTTTCCTCACTCGTGTGGCCAATATGCTGTAATGTCATCACTTATTCAATCCAACGGTTGTAACTTCTCAGCAACCAATCCCCTCCATTTCACACCATCGGATTAGTACTACACAAATCACACTATTATATATAGTAAGTTTGAGCAGAAGCTTGGATATCTGGCAGCAGAAGAACAAGTAGTTGAGAACTAAGAAGGAGAAGCAA以上序列的值应该为多少"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "回答一个数字就可以啦"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "TTTGGTTATTTAATTTTTAAAATCGATCAATTTTTTGGAAAAAATATTTATTTTGTAACGATTTAAAATTATTTAGTCACGAAATATGCAGTTATATATAATATGTATTTTCTTACCATTTAGACTGAATGATTAAATTGAATCGTTTTAAAAAATTAAAGGAT的值为8734.285179"},
    {"role": "user", "content": "TATATATATATATATATATATATATATCAGGCATTTTAAAATAATAAAATAAAACGACTCTTAATTATAATGAAGCATCTCTTTGTTAAACCGGATCTTCCATAATTAATGTAAACATTAATGCTTGATGGAAGTTTTTTGTTCTATCCAATACACGAATGGGTTGTAAATATCTATTTTTATCATTTATGGGTTGTAACTATCAATGCAGTAGTAGTTAGTGTAATGATACGATATATTCTATTGTAGTGTATTATTTTATATAACTATATTCCATTTTAATGTTGGAAATTAAAGACATATGGTTTTATCTCTTGGCAACGAGTCAGAATATATATAATAAAGATAAAATTAGGTCGTAATAATTTTTTTATAATAAATATTTATATACAGTTTCAAGATTTTATTATATTTGTTTGCAAAATTCCCATATAATTGTTTAAGTTTTAAAATTTTCATAATAAATGTTTAAGTTTTCTAAATTCATCCTTATTTGTGAGTAGGGATAAAAATAAGACTAAACCAAACACGACTTTGCTTTAAAAGTCTGGTTTTACCAATCAACCTATTTAAAAAATCTAAAGCACAAGCCAAACCATTTATCTATCACAAGCTTCTTTTTTGGGTTTGTCATGACCGCTTTGAGAGCCTAATTTGAACAAATACTCTATTTAAAAGTTTATTTTAGATTAAAGTTTTGATATAAACTTGACATAACTCTAAAATAGGGTAATCACCGAATAAGTCGATGAGAAAAACTTTATGTAAAATAAATTATAAACTATAAAATTTGTAATCACTACTACTCAAGAGTCAAGACATGAGGTAGAGGTGGAAATAAGCACACTGAGTTTTATTAGAATTAAATGTGTTTATATAATCTAAGCCCATTTTTAGGTCTAATAAGACTCATTTCTTTTATAGGACTAATATGACTATCATGATTATTTATTTAAAAGTCTATTTTACAATAAAGATTTTAATTAAGACAAAATTATTTTTCAATGGCTCAACTTTATATGTAGGTTAACATATTTAATATTTTTTATAAAAAATACGTAGTTTTTTTAAATAAAAATAAATATGTAATTTTATGGTATTATAAAAAAATAAAATTTAATATAATAATAACAACAAAAAATATTAAATATATTTACTTAAATAGGTTGAGCTATTAGGTATAAAAGATTTTTCAAATGTCCTAAAGCTTGACATGCATTTTTAATCAAATTGATTTTTATAAAAACTTTCACCTAATTTATTTATAAAAAAAAAAAAACATCTGGCTTGACCTGTGCTCAGTTAGGCTAGGTCGTAAGTCCTCTCTCGAGCGTCTGATGTATTCTTACCTTATTTGTGAAGAACATAATAAAACTTAATTTTTCAAAACGAAAGGAATAACAATGCTATATATATAATTTACTATTAAATAAAATTTTAAATAATTATCATCCTAATGATGAAACTATTTTTAGGCCAACAATGAATGAGATTAATCTCTGATTCAACGGGTTAAGAAATAATAGTGGTCTTTAATCAGAAAAAAATAACATTTTAAAGGTAACATATAATTGTACTTTTTAATGAATTAAAACTTCAGACATTTTTAAATAATATAATAAAACAACTCTTAATTTTAATGAAGCATCCCTTTGTTAAACCAAATCTTCCATAATGTGAACATTAATGCTTGATGGAAGTTTTTGTTCTATCCAATACACAATGAGTTGTAAATATTTTTTTAATAACATTTATGGGTTGTAAATATGAATGCAGTAGTAATTAGTTTAATGATAAAATATATTCTACTGTATTATTTTCTGTAACTATATTCCATTTTAATGTTTGAAAGACAGATATATTTCCATATATATCTCTTGGCAACTCCTCAGAATATATTATAAAGATGAAAGGTCGTTATAATTTTTTTTAGAATAAATATTTATATACAGTTTCCTAGATTTCGTTATAAAATTCACATATTGAATGAGTATAAATAC的值为7546.179791"},
    {"role": "user", "content": 'CGTATCGATCGCACAAAAACAAAATAATGTAAAATTTGATACAAAGGGTAGAAGACTCCCTTGTATTCTCTTATATTGGTGGCTGCTATAGTTGGACATTAAAAATACAATAAAGTAATGTTTGGATTTTTTTCCTCTTTTGACTTTTCTTTTTATAATTGTCTTTACTGGTACGTTGATTGTAATTTACATTTTGCACTCATATGTAAAACTAAAAGTCATTTTGTCAGTTTTTAAGAGTTTTTTTACTGAATTTTTCTTTTATAATTTTCAATAAAAAAGCTCAAAAACACTTATATCTTATATCCGAACATACCCTTGATTTCCTTGCTACTAATAATACACTTGGAAGCAAATTGCAGTTTAGGCAACTTTGGAGAAGTTGAAAGTGGTTAGAAAGTGGAGAGCCTCATAGAGCTTGCTAGCGGCCAAGGTGGCACAAGAGTGGGGGGAGATTAACTCGTTTTCGTATGAGATGAAAATATGGACCAATGTGGTGTCACTGAGAAGGGGAAAACAATTTCTACCCTGTTAAACCCAGGAGATTGACTCAGAAAATTAAATACTTGCAAGATATAGAGCTAGCTTTTATTGTTCCATAAGAACCATATATGACTTTATCCTAAAAACCACAACGTCTTCAAATTTCCAAAGCCAAACCATTGAAAGCATGAGAAGCCAATCCAGTCTTAATCGGCTAAACCACAAAGGCCACGGTTGTCTTGCATGCACCACACGAAATTGCCTTTTTAAATCTTCAAAAAACAATTGTGTAATTTTATATGATTTACGTAGATTATTTTAGTTATAAGTTAGGAAAATATTTTAAGTTTTTTTATAACAAGTGTTAAGAACATTGATTTAGAAAAAAAAAATATTAAAGAAAAGAAAGCATTTCATTTATAAAGTCATCCAGAAACAAATAAATGATTTATCAATACAAATAAATTGTTGAATTAGTTAATTTAAATTAAAAATATTATGATTTTAATTTTATTCTAGAAATATTGACAAAATTAAATGTTTAAAATTATTGTTATATTTAATGGCATTACTTATAATTCAAAAAATATTTTTTACACAAACAGATATAATTTATATCATCAATAAAACAATGAATAAATACACTTGCATGAAAAATATTGAGTAAGTATTTCGTTAATAAGACAATATATTAAATGAAAATGAAAATTAGCAATTAATATTTTTAAAATGAATCTATATTTTTATATGAAGATTAAAAAATACACTCATATATATGCCATAAGAACCAACAAAAGTACTATTTAAGAAAAGAAAAAAAAAAACCTGCTACATAATTTCCAATCTTGTAGATTTATTTCTTTTATTTTTATAAAGGAGAGTTAAAAAAATTACAAAATAAAAATAGTGAACATCGTCTAAGCATTTTTATATAAGATGAATTTTAAAAATATAATTTTTTTGTCTAAATCGTATGTATCTTGTCTTAGAGCCATTTTTGTTTAAATTGGATAAGATCACACTATAAAGTTCTTCCTCCGAGTTTGATATAAAAAAAATTGTTTCCCTTTTGATTATTGGATAAAATCTCGTAGTGACATTATATTAAAAAAATTAGGGCTCAATTTTTATTAGTATAGTTTGCATAAATTTTAACTTAAAAATAGAGAAAATCTGGAAAAGGGACTGTTAAAAAGTGTGATATTAGAAATTTGTCGGATATATTAATATTTTATTTTATATGGAAACTAAAAAAATATATATTAAAATTTTAAATTCAGAATAATACTTAAATTATTTATTTACTGAAAATGAGTTGATTTAAGTTTTTGAAAAGATGATTGTCTCTTCACCATACCAATTGATCACCCTCCTCCAACAAGCCAAGAGAGACATAAGTTTTATTAGTTATTCTGATCACTCTTCAAGCCTTCTATATAAATAAGTATTGGATGTGAAGTTGTTGCATAACTTGCATTGAACAATTAATAGAAATAACAGAAAAGTAGAAAAGAAAT的值为7405.685612'},
    {"role": "user", "content": query_prompt+format_prompt}
]

In [ ]:
with open('new_rules.txt','r') as f:
    context_list = f.readlines()

In [ ]:
with open('food.txt','r') as f:
    food_list = f.readlines()

In [ ]:
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, ['早上好'], max_new_tokens=1000)

In [ ]:

query_prompt = input()#"我吃的是咖喱饭"
print('患者：')
print(query_prompt)
print('医生：')

bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
format_prompt = "以500千卡为标准判断饮食是否热量超标，如果超标了，就说‘您的饮食热量过高’，并根据食材list给糖尿病患者提一些搭配建议，并给一些具体的搭配例子"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组食材list：" +''.join(food_list)}, #+str(context_list)
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
llm_response = ''
format_prompt = "请根据以上医学标准list，回答患者的问题"
format_prompt = ""
assistant_an = ''
res_messages = [
    {"role": "system", "content": sys_prompt1+',尽量简洁的回复对方，如果对方的给出的信息，不足给出建议要对对方进行提问,然后再给予回答'},
    #{"role": "user", "content": "给定一组医学知识list："+str(context_list)+' 如果问到和血糖相关的问题，请根据规则list给出相应的参考医学知识，不要和前几轮对话输出类似的内容保持多样性'},#不要重复回答问题
    {"role": "user", "content": "给定一组医学标准list："+str(context_list)},#不要重复回答问题
    {"role": "user", "content": "我的血糖是9mol/L，请问有问题不哈"},
    {"role": "assistant", "content": '你现在是餐前还是餐后哈'},
    {"role": "user", "content": "餐后"},
    {"role": "assistant", "content": '根据"餐后血糖应控制在10.0mmol/L以下"这个标准，您的血糖是没有问题的'},
    {"role": "user", "content": "好的，谢谢医生，改天见"},
    {"role": "assistant", "content": '好的，拜拜，祝您生活愉快'},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    #{"role": "user", "content": query_prompt+format_prompt}
]
while True:
    query_prompt = input()
    print(f'\n患者：{query_prompt}')
    #format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
    res_messages.append({"role": "assistant", "content": llm_response})
    res_messages.append({"role": "user", "content": query_prompt+format_prompt}) #+format_prompt
    print('医生：')
    llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组规则list："+str(context_list)},
    #{"role": "assistant", "content": assistant_an},
    #{"role": "user", "content": "给定一组规则list："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    #{"role": "user", "content": query_prompt+format_prompt}
]

query_prompt = ''
print(f'患者：{query_prompt}')
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
res_messages.append({"role": "assistant", "content": llm_response})
res_messages.append({"role": "user", "content": query_prompt+format_prompt})
print(res_messages)
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
llm_response = ''
format_prompt = "请根据以上规则list，请给出对应简短有效的解决方案（细致一些，在3-10条之内），并给出对应的参考规则，同时将其用V1，V2，V3按顺序标号"
assistant_an = ''
res_messages = [
    {"role": "system", "content": sys_prompt1},
    #{"role": "user", "content": "给定一组医学知识list："+str(context_list)+' 如果问到和血糖相关的问题，请根据规则list给出相应的参考医学知识，不要和前几轮对话输出类似的内容保持多样性'},#不要重复回答问题
    {"role": "user", "content": "给定一组医学知识list："+str(context_list)},#不要重复回答问题
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    #{"role": "user", "content": query_prompt+format_prompt}
]
#while True:
query_prompt = '我的血糖不稳定，应该如何运动'
print(f'\n患者：{query_prompt}')
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
res_messages.append({"role": "assistant", "content": llm_response})
res_messages.append({"role": "user", "content": query_prompt+format_prompt}) #+format_prompt
print('医生：')
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000) 

In [ ]:

query_prompt = "我的血糖低了怎么办"#"我吃的是咖喱饭"
print('患者：')
print(query_prompt)
print('医生：')

bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
format_prompt = "并根据以上格式，给出简洁的建议，并将参考规则以V1、V2、V3进行排序附在全部回答的最后面"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": '我的血糖高了怎么办'},
    {"assitant": "user", "content": '1. 监测血糖：\n持续监测餐前餐后血糖，如有不合理的地方及时调整饮食方案。\n每日应至少完成早、中、晚各一次餐前后的血糖配对测量和一次睡前血糖的测量。2.饮食控制：\n饮食中应包含足够的蔬菜和蛋白质，避免食用油炸食品和高糖食品。\n主食多样化，粗粮和细粮1:1搭配，有助于血糖控制。\n每餐主食的量应该控制在2两以内，避免因主食过量造成的大波动。\n餐前餐后血糖值差值波动较大时，建议放慢吃饭速度，增加膳食纤维的食材，注意吃饭顺序，餐后1小时左右运动20-30分钟。\n3.运动:\n患者应每周至少进行150分钟的中等强度运动，或90分钟的高强度运动。\n运动可以增加胰岛素的敏感性，有利于餐后血糖控制。\n\n参考规则：\nV1:持续监测餐前餐后血糖，如有不合理地方及时调整饮食方案\nV2:每日应至少完成早、中、晚各一次餐前后的血糖配对测量和一次睡前血糖的测量\nV3:饮食中应包含足够的蔬菜和蛋白质" 和 "避免食用油炸食品和高糖食品\nV4:主食多样化，粗粮和细粮1:1搭配，有助于血糖控制\nV5:每餐主食的量应该控制在2两以内，避免因主食过量造成的大波动\nV6:患者应每周至少进行150分钟的中等强度运动，或90分钟的高强度运动。\nV7:运动可以增加胰岛素的敏感性，有利于餐后血糖控制'},
    {"role": "user", "content": "给定一组食材list：" + str(context_list)}, #+str(context_list)
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
'1. 监测血糖：\n持续监测餐前餐后血糖，如有不合理的地方及时调整饮食方案。\n每日应至少完成早、中、晚各一次餐前后的血糖配对测量和一次睡前血糖的测量。2.饮食控制：\n饮食中应包含足够的蔬菜和蛋白质，避免食用油炸食品和高糖食品。\n主食多样化，粗粮和细粮1:1搭配，有助于血糖控制。\n每餐主食的量应该控制在2两以内，避免因主食过量造成的大波动。\n餐前餐后血糖值差值波动较大时，建议放慢吃饭速度，增加膳食纤维的食材，注意吃饭顺序，餐后1小时左右运动20-30分钟。\n3.运动:\n患者应每周至少进行150分钟的中等强度运动，或90分钟的高强度运动。\n运动可以增加胰岛素的敏感性，有利于餐后血糖控制。\n\n参考规则：\nV1:持续监测餐前餐后血糖，如有不合理地方及时调整饮食方案\nV2:每日应至少完成早、中、晚各一次餐前后的血糖配对测量和一次睡前血糖的测量\nV3:饮食中应包含足够的蔬菜和蛋白质" 和 "避免食用油炸食品和高糖食品\nV4:主食多样化，粗粮和细粮1:1搭配，有助于血糖控制\nV5:每餐主食的量应该控制在2两以内，避免因主食过量造成的大波动\nV6:患者应每周至少进行150分钟的中等强度运动，或90分钟的高强度运动。\nV7:运动可以增加胰岛素的敏感性，有利于餐后血糖控制'

In [ ]:
query_prompt = "我的血糖高了怎么办"#"我吃的是咖喱饭"
print('患者：')
print(query_prompt)
print('医生：')

bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
format_prompt = "并根据以上格式，给出简洁的建议，并将参考规则以V1、V2、V3进行排序附在全部回答的最后面，同时每条建议后面写上V1、V2、V3"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": '我的血糖低怎么办'},
    {"assitant": "user", "content": '当您的血糖低了（通常指血糖低于3.9mmol/L），您可以采取以下措施：1. **立即补充快速提升血糖的食物**：如含糖饮料、糖果、蜂蜜或白砂糖等，大约15g左右的糖分。例如，可以吃2-3片葡萄糖片或一勺蜂蜜(V1)。\n2. **等待15分钟**：在摄入糖分后，等待15分钟，再次测量血糖，看是否已经恢复到正常范围(V1)。\n3. **如果血糖仍未恢复**：再次摄入15g左右的糖分，并继续监测血糖直到恢复正常(V1)。\n4. **恢复正常后，考虑摄入一些蛋白质和复合碳水化合物**：如一小份坚果和一片全麦面包，以帮助维持血糖水平(V1)。\n\n**注意事项**：\n- 如果您经常出现低血糖，应与医生讨论可能需要调整药物剂量或改变饮食习惯(V2)。\n- 保持与医生的定期联系，及时反馈血糖监测结果和身体状况(V3)。\n\n**参考规则**：\nV1: 出现低血糖后，尽量优先选择糖分高、好吸收的食物，如一勺白砂糖、一勺蜂蜜或者沏到水里喝，如果家里有方糖或葡萄糖片的话，可以吃2-3片，大约15g左右，过15分钟后在复测一下血糖\nV2: 患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况\nV3: 餐后血糖高于20.0mmol/L时，需重新评估饮食和药物治疗方案'},
    {"role": "user", "content": "给定一组食材list：" + str(context_list)}, #+str(context_list)
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

In [ ]:
'当您的血糖低了（通常指血糖低于3.9mmol/L），您可以采取以下措施：1. **立即补充快速提升血糖的食物**：如含糖饮料、糖果、蜂蜜或白砂糖等，大约15g左右的糖分。例如，可以吃2-3片葡萄糖片或一勺蜂蜜(V1)。\n2. **等待15分钟**：在摄入糖分后，等待15分钟，再次测量血糖，看是否已经恢复到正常范围(V1)。\n3. **如果血糖仍未恢复**：再次摄入15g左右的糖分，并继续监测血糖直到恢复正常(V1)。\n4. **恢复正常后，考虑摄入一些蛋白质和复合碳水化合物**：如一小份坚果和一片全麦面包，以帮助维持血糖水平(V1)。\n\n**注意事项**：\n- 如果您经常出现低血糖，应与医生讨论可能需要调整药物剂量或改变饮食习惯(V2)。\n- 保持与医生的定期联系，及时反馈血糖监测结果和身体状况(V3)。\n\n**参考规则**：\nV1: 出现低血糖后，尽量优先选择糖分高、好吸收的食物，如一勺白砂糖、一勺蜂蜜或者沏到水里喝，如果家里有方糖或葡萄糖片的话，可以吃2-3片，大约15g左右，过15分钟后在复测一下血糖\nV2: 患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况\nV3: 餐后血糖高于20.0mmol/L时，需重新评估饮食和药物治疗方案' 


In [ ]:
# 使用内置的 open() 函数来读取文件
file_path = 'rules.txt'  # 替换为你的记事本文件路径

with open(file_path, 'r', encoding='utf-8') as file:
    content_list = file.readlines()

In [ ]:
content_list = [ele.strip() for ele in content_list if len(ele) > 3]